In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_filename = '../input/us-accidents/US_Accidents_Dec20_updated.csv'

## Data preparation and cleaning
- Load the file using Pandas
- Look at some information about the data and the columns
- Fix any missing or incorrect values

In [ ]:
df = pd.read_csv(data_filename)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
numerics = ['int16', 'int32','int64', 'float16','float32','float64']
numeric_df = df.select_dtypes(include=numerics)
len(numeric_df.columns)

### Percentage of missing values per columns

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missing_value = df.isnull().sum().sort_values(ascending=False) /len(df)
missing_percentages = missing_value * 100


In [ ]:
print("Percentages of missing values per columns: \n",missing_percentages)

In [ ]:
missing_percentages[missing_percentages != 0]

## Ploting the percentages

In [ ]:
missing_percentages[missing_percentages != 0].plot(kind='barh')

### Remove columns which are missing above 50% , because 50% of that data will not use in model preparation.

### We see that Number column is missing data closed to 70% so I drop it.

In [ ]:
df = df.drop('Number', axis=1)

In [ ]:
df.shape

Exploratory Analysis and Visualization

The Columns we'll analyze
1. City
2. Start time
3. Start Lat, Start Lng
4. Temparature
5. Weather Condition

In [ ]:
df.columns

In [ ]:
df.City

In [ ]:
cities = df.City.unique()
len(cities)

In [ ]:
cities_by_accident = df.City.value_counts()
cities_by_accident

In [ ]:
# Top 30 cities by number of accident
cities_by_accident[:30]

In [ ]:
cities_by_accident[:25].plot(kind='barh')

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
sns.distplot(cities_by_accident[:100])


In [ ]:
high_accident_cities = cities_by_accident[cities_by_accident >= 1000]
low_accident_cities = cities_by_accident[cities_by_accident < 1000]

In [ ]:
(len(high_accident_cities) /len(cities)) * 100

In [ ]:
sns.histplot(high_accident_cities, log_scale=True)

In [ ]:
(len(low_accident_cities) /len(cities)) * 100

In [ ]:
sns.histplot(low_accident_cities, log_scale=True)

In [ ]:
cities_by_accident[cities_by_accident == 1]

###  Start time

In [ ]:
df.Start_Time[0]

In [ ]:
df.Start_Time = pd.to_datetime(df.Start_Time)

In [ ]:
df.Start_Time

In [ ]:
# to get hour from the timestamp
df.Start_Time.dt.hour

In [ ]:
sns.histplot(df.Start_Time.dt.hour, bins=24)

In [ ]:
sns.histplot(df.Start_Time.dt.dayofweek, bins=7)

### Is the distribution of accidents by hour the same as weekdays as on weekends?

In [ ]:
sunday_start_time = df.Start_Time[df.Start_Time.dt.dayofweek == 6]
sns.histplot(sunday_start_time.dt.hour, bins=24)

On sundays peak accidents occure mostly between 12pm to 11pm

In [ ]:
monday_start_time = df.Start_Time[df.Start_Time.dt.dayofweek == 0]
sns.histplot(monday_start_time.dt.hour, bins=24)

### Month

In [ ]:
sns.histplot(df.Start_Time.dt.month, bins=12)

In [ ]:
# data for yar 2019
df_2019 = df[df.Start_Time.dt.year == 2019]
sns.histplot(df_2019.Start_Time.dt.month, bins=12)

Can you explain the month-wise trenk of accident?
    - Much data are missing before 2016

### Start_lat and Start_lng

In [ ]:
sns.scatterplot(x=df.Start_Lng, y=df.Start_Lat, size=0.001)

In [ ]:
# take smaller sample from the data
sample_df = df.sample(int(0.1 * len(df)))

In [ ]:
sns.scatterplot(x=sample_df.Start_Lng, y=sample_df.Start_Lat, size=0.005)

In [ ]:
# plot that data into map using folium
import folium

In [ ]:
lat, lon = df.Start_Lat[0], df.Start_Lng[0]
lat, lon

In [ ]:
map = folium.Map()
marker = folium.Marker((lat, lon))
marker.add_to(map)
map

## Ask Question and Prepare answer
1. Are there more accidents in warmer or colder areas?
2. Which 5 state has the highest number pf accidents?
3. Does New York show up in the data? If yes, thwn why its count lower regardless of most populated city in US?
4. Among the top 100 cities in number of accident, which state do they belong to most frequently?
5. What time of the day are accidents most frequent in?
    - 6 to 10am and 12pm to 8pm
6. Which months have the most accidents?
7. What is the trend of accidents year over year ( decresing / incresing)?

## Summary and Conclusion

Insights:
- No data from New York
- Less than 3% of cities have more than 1000 yearly accidents.
- Nearly 1200 cities have reported just 1 accidents.
